

# Appendix B: Logistic Regression
(C) 2018 Griffin Chure. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0.](https://creativecommons.org/licenses/by/4.0/) All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pystan
import sys
sys.path.insert(0, '../../')
import mscl.stats
import mscl.mcmc
import mscl.plotting
colors = mscl.plotting.set_plotting_style()
%matplotlib inline

In this notebook, we summarize our model for the estimation of the most-likely parameter values for the logistic regression and demonstrant the implementation using the Stan programming language. 

---

## Channel copy number as a predictor of cell survival

In the [supplemental information](), we discuss in detail our rationale for using logistic regression to compute survival probability. In short, we assume that the log-odds of survial is a linear function of channel copy number,

$$
\log {p_s \over 1 - p_s} = \beta_0 + \beta_1 \log N_c, \tag{1}
$$

where $p_s$ is the probability of survival, $\beta_0$ is the log-odds of survival with one channel, $N_c$ is the number of channels per cell, and $\beta_1$ is the change in the log-odds of survival with a one unit increase in channel copy number. Solving for $p_s$ yields the following,

$$
p_s = {1 \over 1 + N_c^{-\beta_1}e^{-\beta_0}}. \tag{2}
$$

In this notebook, we will cover the implementation of Bayesian parameter estimation for the two coefficients using the [Stan](http://mc-stan.org/) probabilistic programming language.

## Bayesian parameter estimation 

As is discussed in the [supplemental information](), we use a Bayesian definition of probability to estimate the most-likely parameter values of the coefficents shown in Eq 2. In short, we assume that each single-cell observation is a Bernoulli trial with a probability of survival $p_s$. Using Eq. 2 as the functional form of $p_s$, we can write a likelihood for all single-cell observations as,

$$
P(\beta_0, \beta_1\, \vert\, \mathbf{N_c}, \mathbf{s}) = \prod\limits_{i=1}^k \left({1 \over 1 + N_{c_i}^{-\beta_1}e^{-\beta_0}}\right)^s_i\left(1 - {1 \over 1 + N_{c,i}^{-\beta_1}e^{-\beta_0}}\right)^{1 - s_i}, \tag{3}
$$

where $\mathbf{N_c}$ and $\mathbf{s}$ represent the set of channel copy numbers and survival readouts for all cells, and $k$ is the total number of cells in the experiment. This, of course, is assuming that we do have absolute knowledge of the channel copy number. In reality, we have an *estimate* for the copy number, within some reasonable errorbar. In this work, we've chosen to use the statistical error $\sigma_c$ in the conversion from the standard candle reference strain. We can assume that the effective channel copy number is normally distributed about the mean $N_c$. We can define a likelihoood for this parameter as

$$
P(\mathbf{n}\,\vert\, \mathbf{N_c}, \mathbf{\sigma_c}) = {1 \over (2\pi)^{k/2}} \prod\limits_{i=1}^k {1 \over \sigma_{c,i}^2}\exp \left[-{(n_i - N_{c,i})^2 \over 2\sigma_{c,i}^2}\right], \tag{4}
$$

where $n_i$ is the most-likely value of the channel copy number in the $i^{th}$ cell given a measured value $N_{c,i}$ and an error $\sigma_{c,i}$. Combining Eq. 3 and Eq. 4 yields the complete posterior distribution,

$$
P(\beta_0, \beta_1, \mathbf{n}\,\vert \,\mathbf{N_c}, \mathbf{\sigma_c}, \mathbf{s}) = {1 \over (2\pi)^{k/2}}\prod\limits_{i=1}^k {1 \over \sigma_{c,i}^2}\exp\left[ -{(n_i - N_{c,i})^2 \over 2\sigma_{c,i}^2}\right]\left({1 \over 1 + n_i^{-\beta_1}e^{-\beta_0}}\right)^s_i\left(1 - {1 \over 1 + n_i^{-\beta_1}e^{-\beta_0}}\right)^{1 - s_i}. \tag{5}
$$

To evaluate the most-likely paramter values for the coefficients $\beta_0$ and $\beta_1$ as well as the best-estimate channel copy number for each cell $\mathbf{n}$, we used Markov chain Monte Carlo to sample this distribution directly rather than using maximum likelihood estimation or other optimization techniques.

## Defining the model with `Stan` 

While there are enumerable utilities to perform MCMC, we chose to use the `Stan` probabalistic programming language through the python interpreter `pystan`. We chose to use this software as the default sampler is highly efficient and the model definition is very explicit. To set up the model givn in Eq. 5, we wrote a separate script [`logistic_regression.stan`]() which is available on the paper website. Below, we've stated the contents of this separate file along with an explanation of each line. 

```c
data { 
    int<lower=0> J; // Number of distinct data sets
    int<lower=0> N; // Number of data points
    int<lower=0, upper=J> trials[N]; // Sequence of identifiers for each measurement
    vector<lower=0>[N] predictor; // Vector for predictor value
    vector<lower=0>[N] predictor_err; // Statistical error for predictor
    int<lower=0, upper=1> output[N]; // Boolean output vector
}

parameters {
    real beta_0[J]; // Intercept for each trial
    real beta_1[J]; // Slope for each trial
    vector<lower=0>[N] predictor_mu; // The most-likely value for the predictor
}

model {
    // Priors for slope and intercept. Assign weakly informative distributions
    beta_0 ~ normal(0, 100);
    beta_1 ~ normal(0, 100);
    // Compute the most likely value for the predictor value
    predictor_mu ~ normal(predictor, predictor_err);
    // Loop through each trial and compute the likelihood using appropriate params.
    for (i in 1:J) {
        output ~ bernoulli_logit(beta_0[trials[i]] + beta_1[trials[i]] * predictor_mu);
    }
}
```

With this model defined and saved in `code/logistic_regression.stan`, we can load and compile the model using `pystan`. While `pystan` is a useful front-end for interacting with `Stan`, the model must actually be converted to compiled C++ code. In the cell below, we will load in the file. It may take a few minutes to compile, but once compiled we can run the model several times to take advantage of the speed of a compiled program.

In [2]:
# Load in the model using pystan.
model = pystan.StanModel('../stan/logistic_regression.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3fefcfca94943efa94efae991257465e NOW.


With the model compiled, we can begin the sampling. 

## Sampling the posterior distribution

The `model` object has a variety of methods. Of princiipal itnerest to us is the `sampling` function which will perform the high efficiency sampling of the posterior distribution using the "No U-Turns Sampler" (NUTS). To call the sampler, we must first load and massage our experimental data set into a dictionary that is parseable by `Stan`.

In [3]:
# Load the data and restrict it to the shock data set
data = pd.read_csv('../../data/csv/mscl_survival_data.csv')

# Keep only the osmotic shock experimental data.
data = data[data['experiment'] == 'shock'].copy()

In our model, we defined an integer valued vector which contained an identifier for each datum. This identifier will tell STan wheteher it was a "slow" or "fast" osmotic shock. We will insert a column into our DataFrame which contains either a `1` or `2` for slow and fast shocks, respectively. 

In [4]:
# Insert an identifier.
data.loc[:, 'idx'] = 1
data.loc[data['shock_class']=='fast', 'idx'] = 2

We can now define the dictionary that will be fed to the `Stan` sampler. Each key in the dictionary must be exactly as it is defined in the `Stan` model. 

In [ ]:
data_dict = {'J': 2, 'N': len(data), 'trials': data['idx'], 
            'predictor': np.log(data['effective_channels']),
            'predictor_err': np.log(data['effective_channel_err']),
            'output': data['survival'].astype(int)}

With that, we are ready to begin the sampling. We will choose to run a total of 5000 iterations using four chains. This will give us sufficient sampling as well as metric to ensure that we've converged. Sampling in `pystan` is very simple and is just a single line!

In [ ]:
# Sample the distribution.
samples = model.sampling(data=data_dict, iter=5000, chains=4) 

## Extracting the parameter values

With the sampling now complete, we should take a look at some of the statistics to ensure it ran correctly. We can view a buffet of summary statistics by calling `samples` directly. 

In [ ]:
samples

We have a large number of statistics here. You should notice that we have samples for every single cell to determine the most-likely number of channels. The most important of these statistics is the r-hat, which tells us if we have  converged or not. Qualitatively, this value compares the inter-chain variance to the intra-chain variance. If these two values are the same, meaning an r-hat value of `1.0`, the sampler has converged. We see here that we are apparently quite well converged!  To make things easier to handle, we can convert the `samples` object to a pandas DataFrame. To do this, we will used a home-coded function in the `mcmc` module called `chains_to_dataframe`. Please see the documentation for more information.

In [ ]:
# Make the samples a DataFrame keeping only interesting params.
samples_df = mscl.mcmc.chains_to_dataframe(samples, var_names=['beta_0', 'beta_1'])

# Drop the arbitrary `logp` column.
samples_df.drop('logp', axis=1, inplace=True)

# Rename the parameter column names
samples_df.rename(columns={'beta_0__0': 'beta_0_slow', 'beta_0__1':'beta_0_fast',
                          'beta_1__0': 'beta_1_slow', 'beta_1__1':'beta_1_fast'},
                 inplace=True)

In [ ]:
_ = plt.hist(samples_df['beta_1_fast'], bins=100)
_ = plt.hist(samples_df['beta_1_slow'], bins=100)